In [78]:
import numpy as np
import pymc3 as pm
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import datetime, os
import seaborn as sb
import matplotlib.pyplot as plt
# Setup data read from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import data - all data is prenormalized and split
dTrainPath = "//content//drive//My Drive//data//DTrain.csv"
dTestPath = "//content//drive//My Drive//data//DTest.csv"
tTestPath = "//content//drive//My Drive//data//tTest.csv"
tTrainPath = "//content//drive//My Drive//data//tTrain.csv"
dTrain = np.genfromtxt(dTrainPath, delimiter=",")
dTrain =np.transpose(dTrain)
dTest= np.genfromtxt(dTestPath, delimiter=",")
dTest = np.transpose(dTest)
tTrain = np.genfromtxt(tTrainPath, delimiter=",")
tTest = np.genfromtxt(tTestPath, delimiter=",")
#Convert to pandas dataframes
Train = pd.DataFrame(dTrain)
Train.columns = ['SP','N225','SSE','HS','EUR','VIX','T5','T10','T30']
Test = pd.DataFrame(dTest)
Test.columns = ['SP','N225','SSE','HS','EUR','VIX','T5','T10','T30']
tTrain = pd.DataFrame(tTrain)
tTrain.columns = ['y']
tTest = pd.DataFrame(tTest)
tTest.columns = ['y']
#make training dataset
Train['y'] = tTrain
Test['y'] = tTest 

In [0]:
#Setup logistic regression
log_clf = LogisticRegression()
log_clf.fit(dTrain,tTrain)
#Test accuracy as in regression example
tPred = log_clf.predict(dTest)
acc = accuracy_score( tTest, tPred)
print( 'acc = %g' % acc)

In [36]:
with pm.Model() as model:
    pm.glm.GLM.from_formula('y ~ SP + N225 + SSE + HS + EUR + VIX + T5 + T10 + T30', Train) #This is the likelihood of a BLR model
    trace = pm.sample(10000, cores=4,  n_init=100000) #sample 10k times after a burn in of 100k

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sd, T30, T10, T5, VIX, EUR, HS, SSE, N225, SP, Intercept]
Sampling 4 chains:  70%|███████   | 29575/42000 [2:10:44<54:55,  3.77draws/s]  
There were 5998 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.12274851200714122, but should be close to 0.8. Try to increase the number of tuning steps.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [41]:
a=pm.summary(trace) #examine data from output
a

/usr/local/lib/python3.6/dist-packages/pymc3/stats.py:991: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  axis=1, join_axes=[dforg.index])


,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
Intercept,0.621516,0.054007,0.003487,0.504594,0.722799,33.453588,1.066973
SP,-0.131492,0.049085,0.002624,-0.227361,-0.022830,153.006129,1.035836
N225,0.021381,0.039316,0.001815,-0.061652,0.101356,327.014185,1.003437
SSE,-0.021693,0.046981,0.003742,-0.078889,0.063374,3.942641,1.516471
HS,-0.051013,0.041260,0.001756,-0.141202,0.034157,342.830191,1.012035
EUR,0.060316,0.055449,0.003737,-0.022207,0.176709,8.005163,1.233362
VIX,-0.046739,0.038443,0.002285,-0.128969,0.030868,93.289017,1.011083
T5,103.686025,249.133103,24.649746,-394.906826,512.832783,6.514866,1.420630
T10,-103.709634,249.127483,24.649224,-511.512037,396.245849,6.515905,1.420590
T30,-0.013245,0.094739,0.005336,-0.210618,0.154127,14.761827,1.135566


In [74]:
#Extract mean weights
w = a['mean']
w.to_numpy()
wt = w[0:9:1]
#Calculate probabilities
A = np.matmul(wt,np.transpose(dTest))
prob = 1/(1 + np.exp(-A)) 
#Test
tPred = 1*(prob >= 0.5)
testAcc = accuracy_score( tTest, tPred)
testAcc

0.48736664795058954